# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.dummy import DummyClassifier
from IPython.display import display
import warnings


In [2]:
warnings.simplefilter('ignore')
df = pd.read_csv('/datasets/users_behavior.csv')
display(df.sample(10))
display(df.info())

,calls,minutes,messages,mb_used,is_ultra
307,54.0,312.85,10.0,7798.27,0
1312,93.0,688.54,18.0,14117.53,1
561,46.0,333.24,6.0,20286.28,0
2437,45.0,300.97,45.0,17135.44,0
1477,98.0,703.16,4.0,25391.67,1
1623,23.0,169.25,20.0,7476.26,0
2520,73.0,620.25,0.0,21098.49,0
760,28.0,151.92,108.0,20411.22,1
1006,97.0,608.10,50.0,26838.48,1
601,97.0,798.53,75.0,16249.13,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


None

## Разбейте данные на выборки

In [3]:
features = df.drop(['is_ultra'], axis = 1)
target = df['is_ultra']
features_train, features_test, target_train, target_test = train_test_split(features, target, 
                                                    test_size = 0.2, random_state = 42)
features_train, features_valid, target_train, target_valid = train_test_split(features_train, target_train, 
                                                    test_size = 0.33, random_state = 42)
print(features_train.shape , features_valid.shape, features_test.shape , target_train.shape , target_valid.shape 
      , target_test.shape )

(1722, 4) (849, 4) (643, 4) (1722,) (849,) (643,)


Разбиение датасета на тренировочный, валидационный и тестовый происходило в 2 этапа, пропорции выбраны согласно рекомендациям(тренировочный датасет 60% от общего, валидационный и тестовый по 20%)

## Исследуйте модели

Определим, какая модель подойдет лучше всего. Перед нами задача классификации, поэтому в нашем распоряжении три инструмента: дерево классификации, лес классификации и логистическая регрессия. Выясним, какая из моделей после обучения будет иметь меньшую среднеквадратичную ошибку. Для удобства, сразу будем извлекать корень из этого значения.

In [4]:
best_tree_model = None
best_tree_result = 10000
best_tree_depth = 0
for depth in range(1, 16):
    tree_model = DecisionTreeClassifier(random_state = 42, max_depth = depth)
    tree_model.fit(features_train, target_train)
    tree_predictions_valid = tree_model.predict(features_valid)
    tree_result = accuracy_score(target_valid, tree_predictions_valid)
    if tree_result < best_tree_result:
            best_tree_model = tree_model
            best_tree_result = tree_result
            best_tree_depth = depth
print( "Максимальная глубина:", depth, "Лучшая глубина:", best_tree_depth)

Максимальная глубина: 15 Лучшая глубина: 1


Модель дерева показала наилучшую точность при глубине равной 4м, однако значение ошибки достаточно велико.

In [5]:
best_forest_model = None
best_forest_result = 10000
best_forest_est = 0
best_forest_depth = 0
for est in range(10, 51, 5):
    for depth in range (1, 11):
        forest_model = RandomForestClassifier(random_state=42, n_estimators = est, max_depth = depth)
        forest_model.fit(features_train, target_train)
        forest_predictions_valid = forest_model.predict(features_valid)
        forest_result = accuracy_score(target_valid, forest_predictions_valid)
        if forest_result < best_forest_result:
            best_forest_model = forest_model
            best_forest_result = forest_result
            best_forest_est = est
            best_forest_depth = depth
print( "Количество деревьев:", best_forest_est, "Максимальная глубина:", depth,
        "Лучшая глубина:", best_forest_depth)

Количество деревьев: 35 Максимальная глубина: 10 Лучшая глубина: 1


Для модели леса лучшим вариантом оказалась конфигурация с 25ю деревьями и глубиной 8. Ошибка незначительно меньше, чем для модели дерева.

In [6]:
regression_model = LogisticRegression(random_state = 42)
regression_model.fit(features_train, target_train)
regression_predictions_valid = regression_model.predict(features_valid)
regression_result = accuracy_score(target_valid, regression_predictions_valid)
print("Accuracy наилучшей модели на валидационной выборке:", regression_result)

Accuracy наилучшей модели на валидационной выборке: 0.7031802120141343


Модель логичтической регрессии практически не настраиваема, поэтому на выходе получим один результат. Наиболее удачной моделью оказался лес решений. Проверять на тестовой выборке мы будем именно её.

## Проверьте модель на тестовой выборке

In [7]:
forest_model_test_predictions = forest_model.predict(features_test)

def error_count(answers, predictions):
    count = 0
    for i in range(len(answers)):
        if answers[i] != predictions[i]:
            count += 1
    return count

print('Количество ошибок:', error_count(np.array(target_test), np.array(forest_model_test_predictions)))
accuracy = accuracy_score(target_test, forest_model_test_predictions)
print('Точность модели леса решений:', accuracy)

Количество ошибок: 125
Точность модели леса решений: 0.8055987558320373


Лучшая модель совершила 125 ошибок, что соответствует точности примерно 80%. Это хороший результат, с такой моделью можно работать. Однако, для полносты картины сравим точности для остальных моделей, а далее проверим нашу модель на достоверность.

In [8]:
tree_model_test_predictions = tree_model.predict(features_test)
tree_test_result = mean_squared_error(target_test, tree_model_test_predictions)**0.5

def error_count(answers, predictions):
    count = 0
    for i in range(len(answers)):
        if answers[i] != predictions[i]:
            count += 1
    return count

print('Количество ошибок:', error_count(np.array(target_test), np.array(tree_model_test_predictions)))
accuracy = accuracy_score(target_test, tree_model_test_predictions)
print('Точность модели дерева решений:', accuracy)

Количество ошибок: 146
Точность модели дерева решений: 0.7729393468118196


In [9]:
regression_model_test_predictions = regression_model.predict(features_test)
regression_test_result = mean_squared_error(target_test, regression_model_test_predictions)**0.5

def error_count(answers, predictions):
    count = 0
    for i in range(len(answers)):
        if answers[i] != predictions[i]:
            count += 1
    return count

print('Количество ошибок:', error_count(np.array(target_test), np.array(regression_model_test_predictions)))
accuracy = accuracy_score(target_test, regression_model_test_predictions)
print('Точность модели логистической регрессии:', accuracy)

Количество ошибок: 179
Точность модели логистической регрессии: 0.7216174183514774


Как и было замечено ранее, модель дерева и регрессии показали худший результат, по сравнению с моделью леса. К сожалению, логистическая регрессия не прошла через порог, выставленный для отбраковывания нерабочих моделей, так как её точность ниже 75%. Проверим теперь модель леса на адекватность.

## (бонус) Проверьте модели на адекватность

Для проверки на адекватность первым делом сравним точность нашей модели с точностью произвольного классификатора.

In [12]:
dclf = DummyClassifier(strategy='stratified', random_state=42)
dclf.fit(features_test, target_test)

rand_clsfier_predictions = dclf.predict(features_test)

print(dclf.score(rand_clsfier_predictions, target_test))


0.567651632970451


In [13]:
dclf = DummyClassifier(strategy='most_frequent', random_state=42)
dclf.fit(features_test, target_test)

rand_clsfier_predictions = dclf.predict(features_test)

print(dclf.score(rand_clsfier_predictions, target_test))


0.7076205287713841


In [10]:
dclf = DummyClassifier(strategy='constant', random_state=42, constant = 0)
dclf.fit(features_test, target_test)

rand_clsfier_predictions = dclf.predict(features_test)

print(dclf.score(rand_clsfier_predictions, target_test))


0.7076205287713841


При случайном угадывании точно модели всё таки значительно ниже, чем даже у логистической регрессии, однако не равно 50%, несмотря на двоичность нашего класса. Так получается потому, что выборка смещена в сторону значения 0, поэтому простым перебором мы чаще будем попадать в смещенное значение, что увеличит точность. Однако, выбранная модель справляется с прогнозированием гораздо успешнее, что может говорить нам о её достойном качестве. Теперь оценим полноту и точность модели.

В качестве дополнительной самопроверки сравним теперь нашу модель и случайный классификатор, но вместо угадывания будем везде предсказывать ноль.  Проверка показала, что точность таких предсказаний всё-таки ниже пороговой, но уже ощутимо высока. Такое высокое значение упирается в отмеченное нами ранее смещение выборки в сторону нуля. Этот же тезис подтверждается при проверке данным классификатором по стратегии most frequent.

In [11]:
forest_recall = recall_score(target_test, forest_model_test_predictions , labels=None, pos_label=1, average='binary', 
                             sample_weight=None)
forest_precision = precision_score(target_test, forest_model_test_predictions, labels=None, pos_label=1, average='binary', 
                                sample_weight=None)
print('Полнота данных:', forest_recall,' Точность модели:', forest_precision)

Полнота данных: 0.5159574468085106  Точность модели: 0.7404580152671756


Модель далека от идеала, хотя заполняемые данные с почти 75% процентной вероятностью соответствуют реальным значениям(количество истинных положительных результатов отнесенное к количеству истинных срабатываний плюс количество ложных срабатываний). Однако, полнота модели находится на уровне 52%. Получается, примерно половина значений, где дожна была быть 1 была спрогнозированна как 0( полнота определяется как количество истинных положительных результатов отнесенная к количеству истинно положительных результатов плюс количество ложных отрицательных результатов).

**Вывод** Итак, на выходе мы имеем следующий результат: Предложенная модель с 80% точностью посоветует абонентам архивных тарифов верную альтернативу из тарифов актуальных, однако, особенно активным пользователям лишь в 75% процентах случаев будет выгодно обратить внимание на предложенный тариф Ultra, в примерно 50% случаев есть вероятность посоветовать клиенту тариф ниже классом рекомендуемого. С учетом приведенных данных, следует отметить, что при рекомендации того или иного тарифа для перехода стоит обращать внимание пользователей, которым рекомендуется переход на Smart ознакомиться с условиями по тарифу Ultra воизбежание нерационального выбора клиентом.